In [3]:
# Import libraries
import pandas as pd
import json
import sqlite3

# -----------------------------
# Load orders.csv
# -----------------------------
orders = pd.read_csv("orders.csv")

# -----------------------------
# Load users.json
# -----------------------------
with open("users.json", "r") as file:
    users_data = json.load(file)

users = pd.DataFrame(users_data)

# -----------------------------
# Load restaurants.sql
# -----------------------------
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# -----------------------------
# Merge Datasets
# -----------------------------

# Merge orders and users
merged1 = orders.merge(users, on="user_id", how="left")

# Merge with restaurants
final_df = merged1.merge(restaurants, on="restaurant_id", how="left")

# -----------------------------
# 1. Total orders by Gold members
# -----------------------------
gold_orders = final_df[final_df["membership"] == "Gold"]
total_gold_orders = len(gold_orders)

# -----------------------------
# 2. Total revenue from Hyderabad
# -----------------------------
hyd_revenue = final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
hyd_revenue = round(hyd_revenue)

# -----------------------------
# 3. Distinct users
# -----------------------------
distinct_users = final_df["user_id"].nunique()

# -----------------------------
# 4. Average order value for Gold members
# -----------------------------
avg_gold_order = round(gold_orders["total_amount"].mean(), 2)

# -----------------------------
# 5. Orders with rating \u2265 4.5
# -----------------------------
high_rating_orders = final_df[final_df["rating"] >= 4.5]
high_rating_count = len(high_rating_orders)

# -----------------------------
# 6. Top revenue city (Gold only)
# -----------------------------
gold_city_revenue = gold_orders.groupby("city")["total_amount"].sum()

top_city = gold_city_revenue.idxmax()

top_city_orders = len(
    gold_orders[gold_orders["city"] == top_city]
)

# -----------------------------
# Print Results
# -----------------------------

print("1. Total Gold Orders:", total_gold_orders)
print("2. Hyderabad Revenue:", hyd_revenue)
print("3. Distinct Users:", distinct_users)
print("4. Avg Gold Order Value:", avg_gold_order)
print("5. Orders with Rating >= 4.5:", high_rating_count)
print("6. Top Gold Revenue City:", top_city)
print("   Orders in Top City:", top_city_orders)


1. Total Gold Orders: 4987
2. Hyderabad Revenue: 1889367
3. Distinct Users: 2883
4. Avg Gold Order Value: 797.15
5. Orders with Rating >= 4.5: 3374
6. Top Gold Revenue City: Chennai
   Orders in Top City: 1337
